In [7]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [8]:
df = pd.read_csv('workout_data.csv')

In [9]:
df.head(2)

,title,start_time,end_time,description,exercise_title,superset_id,exercise_notes,set_index,set_type,weight_kg,reps,distance_km,duration_seconds,rpe
0,Day 55: Pull,"29 Jun 2024, 10:39","29 Jun 2024, 12:00",NaN,Bent Over Row (Barbell),NaN,NaN,0,normal,60.0,15.0,NaN,NaN,NaN
1,Day 55: Pull,"29 Jun 2024, 10:39","29 Jun 2024, 12:00",NaN,Bent Over Row (Barbell),NaN,NaN,1,normal,60.0,10.0,NaN,NaN,NaN


In [10]:
muscle_group_dict = {
    'Seated Incline Curl (Dumbbell)': 'Arms',
    'Seated Palms Up Wrist Curl': 'Arms',
    'Triceps Extension (Dumbbell)': 'Arms',
    'Preacher Curl (Dumbbell)': 'Arms',
    'Triceps Pushdown': 'Arms',
    'Forearm Dumbell Curl': 'Arms',
    'Hammer Curl (Dumbbell)': 'Arms',
    'Knee Raise Parallel Bars': 'Abs',
    'Leg Press Horizontal (Machine)': 'Legs',
    'Pull Up': 'Back',
    'Seated Row (Machine)': 'Back',
    'Face Pull': 'Shoulders',
    'Leg Extension (Machine)': 'Legs',
    'Dumbell Skiers': 'Shoulders',
    'Shrug (Dumbbell)': 'Back',
    'Seated Cable Row - Bar Grip': 'Back',
    'Lat Pulldown (Machine)': 'Back',
    'Seated Calf Raise': 'Legs',
    'Incline Bench Press (Dumbbell)': 'Chest',
    'Shoulder Press (Dumbbell)': 'Shoulders',
    'Incline Bench Press (Barbell)': 'Chest',
    'Lateral Raise (Dumbbell)': 'Shoulders',
    'Cable Fly Crossovers': 'Chest',
    'Shoulder Press (Machine Plates)': 'Shoulders',
    'Torso Rotation': 'Abs'
}

df['muscle_group'] = df['exercise_title'].map(muscle_group_dict)



In [11]:
df.head(1)
df['start_time'].to_list()[1]
df['start_time'] = pd.to_datetime(df['start_time'], format='%d %b %Y, %H:%M')
df['end_time'] = pd.to_datetime(df['end_time'], format='%d %b %Y, %H:%M')

In [12]:
df.head()

,title,start_time,end_time,description,exercise_title,superset_id,exercise_notes,set_index,set_type,weight_kg,reps,distance_km,duration_seconds,rpe,muscle_group
0,Day 55: Pull,2024-06-29 10:39:00,2024-06-29 12:00:00,NaN,Bent Over Row (Barbell),NaN,NaN,0,normal,60.0,15.0,NaN,NaN,NaN,NaN
1,Day 55: Pull,2024-06-29 10:39:00,2024-06-29 12:00:00,NaN,Bent Over Row (Barbell),NaN,NaN,1,normal,60.0,10.0,NaN,NaN,NaN,NaN
2,Day 55: Pull,2024-06-29 10:39:00,2024-06-29 12:00:00,NaN,Bent Over Row (Barbell),NaN,NaN,2,normal,60.0,10.0,NaN,NaN,NaN,NaN
3,Day 55: Pull,2024-06-29 10:39:00,2024-06-29 12:00:00,NaN,Pull Up,NaN,NaN,0,normal,NaN,6.0,NaN,NaN,NaN,Back
4,Day 55: Pull,2024-06-29 10:39:00,2024-06-29 12:00:00,NaN,Pull Up,NaN,NaN,1,normal,NaN,4.0,NaN,NaN,NaN,Back


In [13]:
# Do repartition plota

df_unique = df.drop_duplicates(subset='title')

minutes = [0] * 24

def calculate_minutes(row):
    global minutes
    start_time = row['start_time']
    end_time = row['end_time']
    for i in range(start_time.hour, end_time.hour + 1):
        if i == start_time.hour:
            minutes[i] += 60 - start_time.minute if start_time.minute != 0 else 0
        elif i == end_time.hour:
            minutes[i] += end_time.minute
        else:
            minutes[i] += 60

df_unique.apply(calculate_minutes, axis=1)
print(minutes)

total_minutes = sum(minutes)
percentage = [(x/total_minutes) * 100 for x in minutes]
fig = go.Figure(data=[go.Bar(
    x = [str(i) for i in range(24)],
    y = percentage
)])

fig.update_layout(title_text='Workout Time Distribution throughout the day',
                xaxis_title='Hour',
                yaxis_title='Percentage of time spent')

[0, 0, 0, 0, 0, 0, 707, 1026, 545, 623, 432, 265, 150, 219, 209, 186, 181, 129, 89, 103, 117, 77, 57, 0]


In [14]:
# Do wykresu liniowego max ciężar (w zależności od powtórzeń w serii (albo bez tego)) w zależności od ćwiczenia

excercise = 'Incline Bench Press (Dumbbell)'

df_excercise = df[df['exercise_title'] == excercise]
df_grouped = df_excercise.groupby('title')[['weight_kg', 'start_time']].max().reset_index()
df_grouped['start_time'][0].date()


datetime.date(2024, 3, 5)

In [15]:
df_weight = pd.read_csv('weight_data.csv')
df_weight.head()
df_weight_cleaned = df_weight.loc[df_weight['Measurement'] == 'Bodyweight']
df_weight_cleaned.head(3)


,Date,Time,Measurement,Value,Unit
0,2019-12-26,22:10:43,Bodyweight,60.0,kgs
1,2019-12-28,22:11:15,Bodyweight,62.6,kgs
2,2021-02-19,06:52:23,Bodyweight,64.9,kgs


In [16]:
fig = px.line(df_weight_cleaned, x='Date', y='Value')
fig

In [17]:
test = df_grouped['start_time'].dt.date
test

0     2024-03-05
1     2024-03-19
2     2024-02-03
3     2024-03-26
4     2024-04-09
5     2024-04-16
6     2024-02-08
7     2024-04-25
8     2024-05-04
9     2024-05-07
10    2024-05-10
11    2024-05-11
12    2024-05-14
13    2024-05-18
14    2024-05-21
15    2024-05-28
16    2024-06-04
17    2024-06-11
18    2024-06-20
19    2024-06-27
20    2024-02-17
21    2024-02-21
22    2024-02-27
Name: start_time, dtype: object

In [19]:
df.groupby(['title', 'duration'])['duration']

KeyError: 'duration'